One thing I found  very useful was learning how to scrape data from websites. This notebook Is intended to show how one can scrape data from the web and clean it so that it can be used for further analysis. I wil be using BeautifulSoup and Pandas library for this tutorial

BeautifulSoup video from edureka for extra resources:

https://www.youtube.com/watch?v=mKxFfjNyj3c

Pandas Dataframes tutorial:

https://pandas.pydata.org/pandas-docs/stable/tutorials.html

Numpy:

http://www.numpy.org/

## Web Scraping 

In [1]:
#importing libraries
import numpy as np
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
#this is where you choose what webpage you want to scrape data from 
url ='http://www.hubertiming.com/results/2018GNG' 

This following step can be important because some websites will need you to specify what user-agent is being used if not the following error will pop up:

HTTP Error 403: Forbidden

Further explanation:

https://stackoverflow.com/questions/13055208/httperror-http-error-403-forbidden

In [3]:
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)

Now that we have the html page we have to create a beautiful soup object with the following function. It's job is to parse the html and break it into python objects.

In [4]:
soup = BeautifulSoup(page,'lxml')
#print(soup) is optional if you want to view the html code of the website

Once you have created a soup object it allows you to extract different information. For example the title.

In [5]:
title = soup.title
print(title)

<title>2018 Give n' Gobble 5K Run Race Results</title>


In [6]:
#exctracting the text
text = soup.get_text()
#print(soup.text) 
#if you want to view the text I ommited it in order to keep this notebook easy to read and follow through

the find_all() method to extract different html tags.

a is for hyperlinks

table is for tables

tr is for table rows

th is for table headers 

td is for table cells

In [7]:
 soup.find_all('a')

[<a class="btn btn-primary btn-lg" href="/results/2018GNG10K" role="button" style="margin: 0px 0px 5px 5px">10K</a>,
 <a class="btn btn-primary btn-lg" href="/results/2018GNG5kWalk" role="button" style="margin: 0px 0px 5px 5px">5K Walk</a>,
 <a href="http://hubertiming.com/">Huber Timing Home</a>,
 <a href="mailto:timing@hubertiming.com">timing@hubertiming.com</a>,
 <a href="#tabs-1" style="font-size: 18px">Results</a>,
 <a name="individual"></a>,
 <a href="http://www.hubertiming.com/"><img height="65" src="/sites/all/themes/hubertiming/images/clockWithFinishSign_small.png" width="50"/>Huber Timing</a>,
 <a href="http://facebook.com/hubertiming/"><img src="/results/FB-f-Logo__blue_50.png"/></a>]

The method above shows all the hyperlinks along with the html tags
so by using a for loop like this one below it will cycle through all of the hyperlinks and only get the hyperlink without the html tag.

In [8]:
for link in soup.find_all("a"):
    print(link.get("href"))

/results/2018GNG10K
/results/2018GNG5kWalk
http://hubertiming.com/
mailto:timing@hubertiming.com
#tabs-1
None
http://www.hubertiming.com/
http://facebook.com/hubertiming/


Now for the important part of scraping data of the web! Converting the tables from the web page into dataframes.

In [9]:
#print out table rows only using the 'tr' argument
rows = soup.find_all('tr')
#print(rows[:10])
# ^ in case you want to see all the rows

In [10]:
for row in rows:
    rowToDF = row.find_all('td')
    
print(rowToDF)
type(rowToDF)

[<td>1067</td>, <td>1207</td>, <td>CADE COLLINS</td>, <td>M</td>, <td>14</td>, <td>BEAVERTON</td>, <td>OR</td>, <td>01:16:45</td>, <td>24:45</td>, <td>527 of 527</td>, <td>M 10-14</td>, <td>86 of 86</td>, <td>00:00:06</td>, <td>01:16:51</td>]


bs4.element.ResultSet

In [11]:
str_cells = str(rowToDF)
cleanText = BeautifulSoup(str_cells, "lxml").get_text()
print(cleanText)

[1067, 1207, CADE COLLINS, M, 14, BEAVERTON, OR, 01:16:45, 24:45, 527 of 527, M 10-14, 86 of 86, 00:00:06, 01:16:51]


In [12]:
#importing the regular expressions module
import re

listRows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean,'',str_cells))
    listRows.append(clean2)
    
print(clean2)
type(clean2)

[1067, 1207, CADE COLLINS, M, 14, BEAVERTON, OR, 01:16:45, 24:45, 527 of 527, M 10-14, 86 of 86, 00:00:06, 01:16:51]


str

In the code above the first step after creating an array listRows is to find all the characters inside the 'td' tags and replace them with an empty string for each table row. The next step is to match what is inside the < bracket followed by anything in between it and the following closing bracket >. 
The re.sub()method is used to find where all the substrings where the regular expressions match and replace them with an empty string. 

In [13]:
#convert the list into a dataframe and get a quick view of the first 10 rows
df1 = pd.DataFrame(listRows)
df1.head(10)

,0
0,"[Finishers:, 1067]"
1,"[Male:, 527]"
2,"[Female:, 540]"
3,[]
4,"[1, 1522, RYAN JONES, M, 24, BEAVERTON, OR, 00..."
5,"[2, 1676, ELI GILBERT, M, 17, COLBERT, WA, 00:..."
6,"[3, 2096, THERESA HAILEY, F, 29, PORTLAND, OR,..."
7,"[4, 1698, THOMAS OSBORNE, M, 17, SHERWOOD, OR,..."
8,"[5, 830, ISMAEL ALVARES, M, 15, SHERWOOD, OR, ..."
9,"[6, 748, JAMES CRABTREE, M, 14, SHERWOOD, OR, ..."


## Data Manipulation and Cleaning

In [14]:
#need to split up the data
df1 = df1[0].str.split(',', expand=True)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,[Finishers:,1067],None,None,None,None,None,None,None,None,None,None,None,None
1,[Male:,527],None,None,None,None,None,None,None,None,None,None,None,None
2,[Female:,540],None,None,None,None,None,None,None,None,None,None,None,None
3,[],None,None,None,None,None,None,None,None,None,None,None,None,None
4,[1,1522,RYAN JONES,M,24,BEAVERTON,OR,00:16:25,05:17,1 of 527,M 20-24,1 of 37,00:00:01,00:16:26]


In [15]:
#takeaway the [ brackets
df1[0] = df1[0].str.strip('[')
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Finishers:,1067],None,None,None,None,None,None,None,None,None,None,None,None
1,Male:,527],None,None,None,None,None,None,None,None,None,None,None,None
2,Female:,540],None,None,None,None,None,None,None,None,None,None,None,None
3,],None,None,None,None,None,None,None,None,None,None,None,None,None
4,1,1522,RYAN JONES,M,24,BEAVERTON,OR,00:16:25,05:17,1 of 527,M 20-24,1 of 37,00:00:01,00:16:26]


In [16]:
#finding the headers for the table
col_labels = soup.find_all('th')

#extract text between html tags  for table headers
Table_Header = []
col_str = str(col_labels)
cleanText2 = BeautifulSoup(col_str,"lxml").get_text()
Table_Header.append(cleanText2)
print(Table_Header)

['[Place, Bib, Name, Gender, Age, City, State, Chip Time, Chip Pace, Gender Place, Age Group, Age Group Place, Time to Start, Gun Time]']


In [17]:
#converting into pandas dataframe
df2 = pd.DataFrame(Table_Header)
df2.head()

,0
0,"[Place, Bib, Name, Gender, Age, City, State, C..."


In [18]:
#splitting them by ','
df2 = df2[0].str.split(',',expand=True)
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,[Place,Bib,Name,Gender,Age,City,State,Chip Time,Chip Pace,Gender Place,Age Group,Age Group Place,Time to Start,Gun Time]


In [19]:
#joining both into one dataframe 
frames = [df2,df1]
df = pd.concat(frames)
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,[Place,Bib,Name,Gender,Age,City,State,Chip Time,Chip Pace,Gender Place,Age Group,Age Group Place,Time to Start,Gun Time]
0,Finishers:,1067],None,None,None,None,None,None,None,None,None,None,None,None
1,Male:,527],None,None,None,None,None,None,None,None,None,None,None,None
2,Female:,540],None,None,None,None,None,None,None,None,None,None,None,None
3,],None,None,None,None,None,None,None,None,None,None,None,None,None
4,1,1522,RYAN JONES,M,24,BEAVERTON,OR,00:16:25,05:17,1 of 527,M 20-24,1 of 37,00:00:01,00:16:26]
5,2,1676,ELI GILBERT,M,17,COLBERT,WA,00:17:04,05:30,2 of 527,M 15-19,1 of 70,00:00:00,00:17:04]
6,3,2096,THERESA HAILEY,F,29,PORTLAND,OR,00:17:10,05:32,1 of 540,F 25-29,1 of 63,00:00:01,00:17:11]
7,4,1698,THOMAS OSBORNE,M,17,SHERWOOD,OR,00:17:24,05:37,3 of 527,M 15-19,2 of 70,00:00:01,00:17:25]
8,5,830,ISMAEL ALVARES,M,15,SHERWOOD,OR,00:17:27,05:37,4 of 527,M 15-19,3 of 70,00:00:00,00:17:27]


In [20]:
#how to assign the first row to be the table header
df = df.rename(columns=df.iloc[0])
df.head()

,[Place,Bib,Name,Gender,Age,City,State,Chip Time,Chip Pace,Gender Place,Age Group,Age Group Place,Time to Start,Gun Time]
0,[Place,Bib,Name,Gender,Age,City,State,Chip Time,Chip Pace,Gender Place,Age Group,Age Group Place,Time to Start,Gun Time]
0,Finishers:,1067],None,None,None,None,None,None,None,None,None,None,None,None
1,Male:,527],None,None,None,None,None,None,None,None,None,None,None,None
2,Female:,540],None,None,None,None,None,None,None,None,None,None,None,None
3,],None,None,None,None,None,None,None,None,None,None,None,None,None


In [21]:
#clean the table and drop any missing values
df = df.dropna(axis=0, how='any')

In [22]:
df = df.drop(df.index[0])
df.head()

,[Place,Bib,Name,Gender,Age,City,State,Chip Time,Chip Pace,Gender Place,Age Group,Age Group Place,Time to Start,Gun Time]
4,1,1522,RYAN JONES,M,24,BEAVERTON,OR,00:16:25,05:17,1 of 527,M 20-24,1 of 37,00:00:01,00:16:26]
5,2,1676,ELI GILBERT,M,17,COLBERT,WA,00:17:04,05:30,2 of 527,M 15-19,1 of 70,00:00:00,00:17:04]
6,3,2096,THERESA HAILEY,F,29,PORTLAND,OR,00:17:10,05:32,1 of 540,F 25-29,1 of 63,00:00:01,00:17:11]
7,4,1698,THOMAS OSBORNE,M,17,SHERWOOD,OR,00:17:24,05:37,3 of 527,M 15-19,2 of 70,00:00:01,00:17:25]
8,5,830,ISMAEL ALVARES,M,15,SHERWOOD,OR,00:17:27,05:37,4 of 527,M 15-19,3 of 70,00:00:00,00:17:27]


When renaming the columns its important to check if there are any spaces between the words since it might not execute properly if it is not exact

In [23]:
df.rename(columns={'[Place': 'Place'},inplace=True)
df.rename(columns={' Gun Time]': 'GunTime'},inplace=True)
df.head()

,Place,Bib,Name,Gender,Age,City,State,Chip Time,Chip Pace,Gender Place,Age Group,Age Group Place,Time to Start,GunTime
4,1,1522,RYAN JONES,M,24,BEAVERTON,OR,00:16:25,05:17,1 of 527,M 20-24,1 of 37,00:00:01,00:16:26]
5,2,1676,ELI GILBERT,M,17,COLBERT,WA,00:17:04,05:30,2 of 527,M 15-19,1 of 70,00:00:00,00:17:04]
6,3,2096,THERESA HAILEY,F,29,PORTLAND,OR,00:17:10,05:32,1 of 540,F 25-29,1 of 63,00:00:01,00:17:11]
7,4,1698,THOMAS OSBORNE,M,17,SHERWOOD,OR,00:17:24,05:37,3 of 527,M 15-19,2 of 70,00:00:01,00:17:25]
8,5,830,ISMAEL ALVARES,M,15,SHERWOOD,OR,00:17:27,05:37,4 of 527,M 15-19,3 of 70,00:00:00,00:17:27]


In [24]:
df['GunTime'] = df['GunTime'].str.strip(']')
df.head()

,Place,Bib,Name,Gender,Age,City,State,Chip Time,Chip Pace,Gender Place,Age Group,Age Group Place,Time to Start,GunTime
4,1,1522,RYAN JONES,M,24,BEAVERTON,OR,00:16:25,05:17,1 of 527,M 20-24,1 of 37,00:00:01,00:16:26
5,2,1676,ELI GILBERT,M,17,COLBERT,WA,00:17:04,05:30,2 of 527,M 15-19,1 of 70,00:00:00,00:17:04
6,3,2096,THERESA HAILEY,F,29,PORTLAND,OR,00:17:10,05:32,1 of 540,F 25-29,1 of 63,00:00:01,00:17:11
7,4,1698,THOMAS OSBORNE,M,17,SHERWOOD,OR,00:17:24,05:37,3 of 527,M 15-19,2 of 70,00:00:01,00:17:25
8,5,830,ISMAEL ALVARES,M,15,SHERWOOD,OR,00:17:27,05:37,4 of 527,M 15-19,3 of 70,00:00:00,00:17:27


In [25]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1067 entries, 4 to 1070
Data columns (total 14 columns):
Place               1067 non-null object
 Bib                1067 non-null object
 Name               1067 non-null object
 Gender             1067 non-null object
 Age                1067 non-null object
 City               1067 non-null object
 State              1067 non-null object
 Chip Time          1067 non-null object
 Chip Pace          1067 non-null object
 Gender Place       1067 non-null object
 Age Group          1067 non-null object
 Age Group Place    1067 non-null object
 Time to Start      1067 non-null object
GunTime             1067 non-null object
dtypes: object(14)
memory usage: 125.0+ KB


Now we went from a table seen on the 2018 Give n' Gobble 5K Run for the time and racers info to an organized and cleaned dataframe.